In [ ]:
try:
    with open("../global_setup.py") as setupfile:
        exec(setupfile.read())
except FileNotFoundError:
    print('Setup already completed')

In [ ]:
import pprint

In [ ]:
from src.text.document_retrieval.wikipedia import Wikipedia # Generic Wikipedia class
wikipedia = Wikipedia(
    language="Danish",
    cache_directory_url=False
)

In [ ]:
import re
# ESA relatedness package
from sklearn.feature_extraction.text import TfidfVectorizer

class Rsspedia:
    def __init__(self, wikipedia: Wikipedia):

        self.wikipedia = wikipedia

        self.search_results = []
        self.content = self.texts = self.wikipedia_results = None
        
        self._transformer = TfidfVectorizer(stop_words = None, norm = "l2", use_idf = True, sublinear_tf = False)
        #self._Y = self._transformer.fit_transform(texts)

    def loadTexts(self, texts):
        self.texts = texts

    def display_beautifully(self, titles, texts, urls):
        formatted_result_list = ["<ol>"]
        for i in range(len(titles)):
            formatted_result = "\n".join([
                "<li>",
                f"<p><a href=\"{urls[i]}\">{titles[i]}</a></p>",
                f"<p>{texts[i]}</p>",
                "</li>"
            ])
            formatted_result_list.append(formatted_result)
        formatted_result_list.append("</ol>")
        formatted_results = "\n".join(formatted_result_list)
        return formatted_results

    def search_wiki(self, search_texts, n_matches = 3, search_type = "okapibm25"):
        
        titles = [] 
        texts = []
        urls = []
        
        # (1) Remove unnecessary symbols from the search text
        pattern = re.compile('[^a-zA-Z0-9åÅøØæÆ ]+', re.UNICODE)

        if search_texts:
            for i, text in enumerate(search_texts):
                # (1) Remove unnecessary symbols from the search text
                text = pattern.sub('', text)
                
                if search_type == "okapibm25":
                    wikipedia_results, search_terms = self.wikipedia.search(query = text, k_1 = 1.2, b = 0.75)
                else if search_type = "esa_relatedness":
                    
                    y = self._transformer.transform([text])
                    D = np.array((self._Y * y.T).todense())
                    indices = np.argsort(-D, axis=0)
                    titles = [self._titles[index] for index in indices[:n_matches, 0]]
                    texts = [self._Y[index] for index in indices[:n_matches, 0]]
                
                for index, score in wikipedia_results[:n_matches].items():
                    document = self.wikipedia.documents[index]
                    titles.append(document.title)
                    texts.append(document.abstract)
                    urls.append(document.url)

        return titles, texts, urls

rsspedia = Rsspedia(wikipedia)
titles, texts, urls = rsspedia.search_wiki(["Professorer om oprettelsen af Skat: 'Utrolig løsagtigt' og 'frygtelig forhastet'"])
pprint.pprint(titles)

In [ ]:
print(wikipedia.documents[1])